In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv("train.csv")

print("setup complete")

setup complete


In [55]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
print("lib loaded")

lib loaded


In [57]:
df.drop(["PassengerId", "Name", "Ticket"], axis = 1, inplace=True) # highly cardinal columns
X = df.drop("Survived", axis = 1)
y = df["Survived"]

In [58]:
categorical = [col for col in X.columns if(df[col].dtypes == "O")]
numerical = [col for col in X.columns if(df[col].dtypes != "O")]
X['Cabin'] = X['Cabin'].str[0]

In [59]:

numerical_transformer = Pipeline(
        steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
    ])
categorical_transformer1 = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = "most_frequent")),
    ('onehot', OneHotEncoder(handle_unknown = "ignore")),
])

categorical_transformer2 = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = "most_frequent")),
    ('woe', ce.HelmertEncoder()),
])

preprocessor = ColumnTransformer(transformers = [
    ('num', numerical_transformer, numerical),
    ('cat1', categorical_transformer1, ['Sex', 'Embarked']),
    ('cat2', categorical_transformer2, ['Cabin'])
])

In [60]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.75, random_state=1)

In [61]:
import warnings
warnings.filterwarnings('ignore')


In [72]:
def check(a, b):
        model = SVC(kernel = 'rbf', C = a, gamma = b)
        my_pipeline = Pipeline(steps = [
                ('preprocessor', preprocessor),
                ('model', model)
        ])
        my_pipeline.fit(X_train, y_train)
        y_pred = my_pipeline.predict(X_valid)
        print('C = ', a, ' gamma = ', b, " accuracy ", accuracy_score(y_pred, y_valid))

In [73]:
C =  [0.1, 1, 10, 100, 1000]
gamma = [1, 0.1, 0.01, 0.001, 0.0001]
for a in C:
    for b in gamma:
        check(a, b)


C =  0.1  gamma =  1  accuracy  0.5964125560538116
C =  0.1  gamma =  0.1  accuracy  0.6098654708520179
C =  0.1  gamma =  0.01  accuracy  0.5964125560538116
C =  0.1  gamma =  0.001  accuracy  0.5964125560538116
C =  0.1  gamma =  0.0001  accuracy  0.5964125560538116
C =  1  gamma =  1  accuracy  0.772795216741405
C =  1  gamma =  0.1  accuracy  0.7817638266068759
C =  1  gamma =  0.01  accuracy  0.71898355754858
C =  1  gamma =  0.001  accuracy  0.5964125560538116
C =  1  gamma =  0.0001  accuracy  0.5964125560538116
C =  10  gamma =  1  accuracy  0.772795216741405
C =  10  gamma =  0.1  accuracy  0.7817638266068759
C =  10  gamma =  0.01  accuracy  0.7802690582959642
C =  10  gamma =  0.001  accuracy  0.7204783258594918
C =  10  gamma =  0.0001  accuracy  0.5964125560538116
C =  100  gamma =  1  accuracy  0.7668161434977578
C =  100  gamma =  0.1  accuracy  0.7653213751868461
C =  100  gamma =  0.01  accuracy  0.7787742899850523
C =  100  gamma =  0.001  accuracy  0.7832585949177877

In [74]:
model = SVC(kernel = 'rbf', C = 1000, gamma = 0.01)
my_pipeline = Pipeline(steps = [
                ('preprocessor', preprocessor),
                ('model', model)
        ])
my_pipeline.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat1',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown=

In [75]:
test_df = pd.read_csv("test.csv")

In [76]:
psgId = test_df["PassengerId"]
test_df.drop(["PassengerId", "Name", "Ticket"], axis = 1, inplace=True)
test_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,34.5,0,0,7.8292,NaN,Q
1,3,female,47.0,1,0,7.0000,NaN,S
2,2,male,62.0,0,0,9.6875,NaN,Q
3,3,male,27.0,0,0,8.6625,NaN,S
4,3,female,22.0,1,1,12.2875,NaN,S


In [77]:
final_model = RandomForestClassifier(n_estimators = 400,max_leaf_nodes = 29, random_state = 1, n_jobs = -1)
final_pipeline = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('model', final_model)
])

final_pipeline.fit(X, y)
y_pred = final_pipeline.predict(test_df)

In [78]:
res = pd.DataFrame({
    'PassengerId' : psgId,
    'Survived' : y_pred
})

In [79]:
res.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [83]:
res.to_csv("submission4.0.csv", index=False)

In [84]:
check = pd.read_csv("submission4.0.csv")
check.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
